In [106]:
!pip install pandas python-docx
!pip install pywin32

In [188]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from datetime import datetime
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import numpy as np
import os

#def replace_placeholder_in_runs(paragraph, placeholder, new_text):
#    # Combine all runs in the paragraph into a single string
#    full_text = ''.join(run.text for run in paragraph.runs)
#
#    # If the placeholder is not in the full text, there's nothing to replace
#    if placeholder not in full_text:
#        return
#
#    # Replace the placeholder in the full text
#    full_text = full_text.replace(placeholder, new_text)
#
#    # Clear all runs in the paragraph
#    for run in paragraph.runs:
#        run.clear()
#
#    # Add a new run with the modified text
#    paragraph.add_run(full_text)
    
# Load Excel data
#MyFile = input("Enter your file path: ")
df = pd.read_excel('PayMarch2024.xlsx')

# Start Word
MonthsNames = ['ינואר','פברואר','מרץ','אפריל','מאי','יוני','יולי','אוגוסט','ספטמבר','אוקטובר','נובמבר','דצמבר']

# Loop over each row in the Excel data
FirstPayer = 1
FirstMonth = 3
LastPayer = 34

Length_of_Month_Vec = 13-FirstMonth

Payers = (np.array([4,12,15,25])-1)*2 #range(FirstPayer-1, 2*LastPayer-1, 2)
for i in Payers:  # size(raw, 1)
    # collect date
    Name = df.iloc[i, 0] # take the name
    Apartment = int(df.iloc[i, 1]) # take the Ap.No.
    Methodi = df.iloc[i+1, 2]
    if pd.isnull(Methodi):
        Methodi = 'העברה'
    paymentsVec = df.iloc[i, (2+FirstMonth):(15)].values # start at the FirstMonth
    payments = paymentsVec[~pd.isnull(paymentsVec)] #only valid paid
    Sum = sum(payments) #total paid by payer
    paidMonths = round(Sum/250) #relevant paid month at all
    
    #try:
    #    DateVector = [parse_date(x) if isinstance(x, str) else None for x in df.iloc[i+1, 3:15]]
    #except:
    #adding year to string for better date string recognition
    DateVector = [datetime.strptime(x+'.2024', 'שולם %d.%m.%Y') if isinstance(x, str) else None for x in df.iloc[i+1, (2+FirstMonth):15]]
    
    ValiDate = [x is not None for x in DateVector]

    PaidVec = [False]*Length_of_Month_Vec
    PaidVec[:paidMonths] = [True]*paidMonths
    PaidMonthVec = [0]*Length_of_Month_Vec
    Last_i = 0
    for Vi in range(len(ValiDate)):
        if ValiDate[Vi]:
            nowNumMonth = round(paymentsVec[Vi]/250)
            if nowNumMonth > 0:
                PaidMonthVec[Last_i:Last_i+nowNumMonth] = [Vi]*(nowNumMonth)
                Last_i = Last_i + nowNumMonth
            else:
                Last_i = Last_i + 1

    monthStr = '{:02d}'.format(paidMonths)
    monthNumStr = '{:02d}'.format(max([PaidMonthVec.index(x)+1 for x in PaidMonthVec if x != 0] + [FirstMonth]))
    #monthNumStr = '{:02d}'.format(max([PaidMonthVec.index(x) for x in PaidMonthVec if x != 0, FirstMonth]))
    yearStr = '24'
    receiptNumber = '{:02d}'.format(int(Apartment)) + monthStr + monthNumStr + yearStr

    # Open the template document
    document = Document(os.path.join(os.getcwd(), 'template.docx'))

    # Replace placeholders with data from Excel
    #for paragraph in document.paragraphs:
    #    replace_placeholder_in_runs(paragraph, '{Name}', Name)
    #    replace_placeholder_in_runs(paragraph, '{Apartment}', str(Apartment))
    #    replace_placeholder_in_runs(paragraph, '{Date}', datetime.now().strftime('%d/%m/%Y'))
    #    replace_placeholder_in_runs(paragraph, '{Receipt}', receiptNumber)
    # Replace placeholders with data from Excel
    for p in document.paragraphs:
        if '{Name}' in p.text:
            p.text = p.text.replace('{Name}', Name)
        if '{Apartment}' in p.text:
            p.text = p.text.replace('{Apartment}', str(Apartment))
        if '{Date}' in p.text:
            p.text = p.text.replace('{Date}', datetime.now().strftime('%d/%m/%Y'))
        if '{Receipt}' in p.text:
            p.text = p.text.replace('{Receipt}', receiptNumber)


    # Get the existing table
    table = document.tables[0]

    # Fill table with data
    index = 0
    for Vi in range(1, len(ValiDate)):
        if ValiDate[Vi]:
            #try:
            AllNames = [MonthsNames[x] for x in PaidMonthVec if x == Vi]
            monthString = ' - '.join(list(set([AllNames[0], AllNames[-1]])))
            #except:
            #    continue
            index = index + 1
            row = table.add_row()
            row.cells[0].text = str(index)
            row.cells[1].text = 'תשלום עבור ' + monthString + ' 2024'
            row.cells[2].text = Methodi
            row.cells[3].text = DateVector[Vi].strftime('%d/%m/%y')
            row.cells[4].text = str(paymentsVec[Vi]) + ' ₪'

    # Replace {Sum} placeholder with sum of amounts
    for p in document.paragraphs:
        if '{Sum}' in p.text:
            p.text = p.text.replace('{Sum}', str(Sum))

    # Save the new document
    document.save(os.path.join(os.getcwd(), 'קבלה ' + receiptNumber + ' ועד בית.docx'))
    os.system('pandoc -s "קבלה ' + receiptNumber + ' ועד בית.docx" -o "קבלה ' + receiptNumber + ' ועד בית.pdf"')



   # def parse_date(x):
   #     try:
   #         return datetime.strptime(x, 'שולם %d.%m')
   #     except ValueError:
   #         # Handle February 29 dates
   #         return datetime.strptime(x+'.2024', 'שולם %d.%m.%Y')

In [192]:
Name

'איתן'

In [312]:
import pandas as pd
import win32com.client as win32
from datetime import datetime
import os

# Load Excel data
MyFile = "PayMarch2024.xlsx" # input("Enter your file path: ")
df = pd.read_excel(MyFile)

# Start Word
word = win32.gencache.EnsureDispatch('Word.Application')
word.Visible = True

MonthsNames = ['ינואר','פברואר','מרץ','אפריל','מאי','יוני','יולי','אוגוסט','ספטמבר','אוקטובר','נובמבר','דצמבר']

# Loop over each row in the Excel data
FirstMonth = 3
Length_of_Month_Vec = 13-FirstMonth
FirstPayer = 1
LastPayer = 25

Payers = (np.array([4,12,15,25])-1)*2 #range(2*(FirstPayer-1), 2*LastPayer-1, 2)
for i in Payers:  # size(raw, 1)
    # collect date
    Name = df.iloc[i, 0]
    Apartment = int(df.iloc[i, 1])
    Methodi = df.iloc[i+1, 2]
    if pd.isnull(Methodi):
        Methodi = 'העברה'
    paymentsVec = df.iloc[i, 3:15].values
    payments = paymentsVec[~pd.isnull(paymentsVec)]
    Sum = sum(payments)
    paidMonths = round(Sum/250)
    DateVector = [datetime.strptime(x+'.24', 'שולם %d.%m.%y') if isinstance(x, str) else None for x in df.iloc[i+1, 3:15]]
    ValiDate = [x is not None for x in DateVector]

    PaidVec = [False]*12
    PaidVec[:paidMonths] = [True]*paidMonths
    PaidMonthVec = [0]*12
    Last_i = 0
    for Vi in range(1,len(ValiDate)+1):
        if ValiDate[Vi-1]:
            nowNumMonth = round(paymentsVec[Vi-1]/250)
            if nowNumMonth > 0:
                PaidMonthVec[Last_i:Last_i+nowNumMonth] = [Vi]*(nowNumMonth)
                Last_i = Last_i + nowNumMonth
            else:
                Last_i = Last_i + 1

    RelevantPaid = payments[FirstMonth-1:]
    RelevantSum  = sum(RelevantPaid)
    if RelevantSum<1:
        continue
    RelevandPaidMonths = round(RelevantSum/250) 
    monthStr = '{:02d}'.format(RelevandPaidMonths)
    monthNumStr = '{:02d}'.format(max([min([PaidMonthVec.index(x)+1 for x in PaidMonthVec if x != 0]),FirstMonth]))
    yearStr = DateVector[ValiDate.index(True)].strftime('%y')
    receiptNumber = '{:02d}'.format(Apartment) + monthStr + monthNumStr + yearStr

    # Open the template document
    doc = word.Documents.Open(os.path.join(os.getcwd(), 'template.docx'))

    # Save the new document
    doc.SaveAs(os.path.join(os.getcwd(), 'קבלה ' + receiptNumber + ' ועד בית.docx'))

    # Replace placeholders with data from Excel
    word.Selection.Find.Execute('{Name}', False, False, False, False, False, True, 1, True, Name, 2)
    word.Selection.Find.Execute('{Apartment}', False, False, False, False, False, True, 1, True, str(Apartment), 2)
    word.Selection.Find.Execute('{Date}', False, False, False, False, False, True, 1, True, datetime.now().strftime('%d/%m/%Y'), 2)
    word.Selection.Find.Execute('{Receipt}', False, False, False, False, False, True, 1, True, receiptNumber, 2)

    # Get the existing table
    table = doc.Tables(1)

    # Fill table with data
    index = 0
    for Vi in range(FirstMonth, len(ValiDate)+1):
        if ValiDate[Vi-1]:
            try:
                AllNames = [MonthsNames[ind] for ind,x in enumerate(PaidMonthVec) if x == Vi]
                monthString = ' - '.join(list(set([AllNames[-1], AllNames[0]])))
            except:
                continue
            index = index + 1
            row = table.Rows.Add()
            row.Cells(1).Range.Text = str(index)
            row.Cells(2).Range.Text = 'תשלום עבור ' + monthString + ' 2024'
            row.Cells(3).Range.Text = Methodi
            row.Cells(4).Range.Text = DateVector[Vi-1].strftime('%d/%m/%y')
            row.Cells(5).Range.Text = str(paymentsVec[Vi-1]) + ' ₪'

    # Replace {Sum} placeholder with sum of amounts
    word.Selection.Find.Execute('{Sum}', False, False, False, False, False, True, 1, True, str(Sum), 2)

    doc.Save()
    doc.Close()

word.Quit()


i

In [300]:
max([min([PaidMonthVec.index(x)+1 for x in PaidMonthVec if x != 0]),FirstMonth])

1

In [307]:
sum(payments[7:])

0

In [292]:
Vi=2

In [50]:
DateVector = [datetime.strptime(x+'.2024', 'שולם %d.%m.%Y') if isinstance(x, str) else None for x in df.iloc[i+1, 3:15]]
    

ValueError: time data 'שולם11.2.2024' does not match format 'שולם %d.%m.%Y'

In [64]:
dateind = 0;
NowVec = [datetime.strptime('שולם '+'1.1.2024', 'שולם %d.%m.%Y')]
for x in df.iloc[i+1, 3:15]:
    if isinstance(x, str):
        try:
            NowVec=datetime.strptime(x+'.2024', 'שולם %d.%m.%Y')
        except:
            NowVec=datetime.strptime(x+'.2024', 'שולם%d.%m.%Y')
    else:
        NowVec+=None
    dateind+=1
NowVec

ValueError: time data 'שולם 10.1.2024' does not match format 'שולם%d.%m.%Y'

In [63]:
NowVec

[datetime.datetime(2024, 1, 10, 0, 0)]